In [90]:
import mysql.connector as connection
import pandas as pd
import numpy as np

In [5]:
# Establish connection with MySQL DB
mydb = connection.connect(host="localhost", port="3306", user="root", password="", database="food_ordering_db")

In [50]:
# Accessing food table
query = """Select *
						FROM `food`;"""
food = pd.read_sql(query,mydb)
print(food.head())

# Accessing user table
query = """Select *
						FROM `user`;"""
users = pd.read_sql(query,mydb)
print(users.head())

   foodID  restaurantID                      foodName  price
0       1             1  Mala Grilled Burmese Chicken  10000
1       2             1             Mala French Fries   3500
2       3             1         Rakhine Seafood Salad   6000
3       4             1           Steamed Chicken Leg   5000
4       5             1            Mala Grilled Prawn  12000
   userID firstName lastName
0       1     James    Smith
1       2   Michael    Smith
2       3    Robert    Smith
3       4     David    Smith
4       5     James  Johnson


In [11]:
# Accessing userID and foodID data with ratings
query = """Select o.orderID, o.foodID, c.userID 
						FROM `order` o, `cart` c
						WHERE o.cartID = c.cartID;"""
orders = pd.read_sql(query,mydb)
print(orders.head())
print(list(orders.columns))

   orderID  foodID  userID
0        1    3600      80
1        2     461     191
2        3     858     315
3        4    6772     269
4        5    3112     264
['orderID', 'foodID', 'userID']


In [95]:
# Merge order and food queries
combine_food_order = pd.merge(orders, food, on="foodID")
combine_food_order.head()

,orderID,foodID,userID,restaurantID,foodName,price
0,1,3600,80,119,Pounded Rice Noodle,1500
1,2,461,191,16,Maocai,5000
2,3,858,315,38,ပြည်ကြီးငါးသုပ် ယိုးဒယားစတိုင်လ်,3000
3,4,6772,269,220,Fresh Springrolls With Shrimp,4300
4,5,3112,264,103,Mixed Grills Mala Xiang Guo,3000


In [99]:
print(f'Amount of unique foodIDs - {len(pd.unique(combine_food_order["foodID"]))}')
print(f'Amount of unique userIDs - {len(pd.unique(combine_food_order["userID"]))}')

Amount of unique foodIDs - 179
Amount of unique userIDs - 77


In [84]:
with open("combined_food_order" + ".csv", "w") as f:
	f.write(combine_food_order.to_csv())

Getting the number of times food was ordered

In [77]:
# combine_food_order = combine_food_order.dropna(axis = 0, subset = ['bookTitle'])
food_order_count = (combine_food_order.
     groupby(by = ["foodID"]).
     count().
     reset_index().
     sort_values(by = ["orderID"]).
     rename(columns = {"orderID": "totalCount"})
     [["foodID", "totalCount"]]
     )
food_order_count.tail()

,foodID,totalCount
93,4385,2
63,2996,2
90,4030,3
73,3255,4
10,464,5


In [78]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(food_order_count["totalCount"].describe())

count   179.000
mean      1.117
std       0.465
min       1.000
25%       1.000
50%       1.000
75%       1.000
max       5.000
Name: totalCount, dtype: float64


In [85]:
order_with_total_count = combine_food_order.merge(food_order_count, left_on = 'foodID', right_on = 'foodID', how = 'left')
order_with_total_count["has_ordered"] = 1
order_with_total_count.loc[order_with_total_count["foodName"] == "Water"]
# order_with_total_count.head()

,orderID,foodID,userID,restaurantID,foodName,price,totalCount,has_ordered
9,9,464,191,16,Water,500,5,1
10,68,464,191,16,Water,500,5,1
11,102,464,191,16,Water,500,5,1
12,104,464,191,16,Water,500,5,1
13,190,464,191,16,Water,500,5,1


In [83]:
with open("abcde" + ".csv", "w") as f:
	f.write(order_with_total_count.to_csv())

In [132]:
from scipy.sparse import csr_matrix
order_with_total_count = order_with_total_count.drop_duplicates(['userID', 'foodName'])
order_with_total_count_pivot = order_with_total_count.pivot(index = 'foodName', columns = 'userID', values = 'has_ordered').fillna(0)
order_with_total_count_matrix = csr_matrix(order_with_total_count_pivot.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(order_with_total_count_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [136]:
# query_index = np.random.choice(order_with_total_count_pivot.shape[0])
# print(query_index)
query_index = 153
distances, indices = model_knn.kneighbors(order_with_total_count_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

order_with_total_count_pivot.iloc[query_index,:].values.reshape(1,-1)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [137]:
order_with_total_count_pivot.index[query_index]

'Tiramisu'

In [138]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(order_with_total_count_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, order_with_total_count_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Tiramisu:

1: Kaw Yay Noodle, with distance of 0.0:
2: Chicken Basil Rice, with distance of 0.0:
3: ကြက်ချဉ်စပ်ထမင်း, with distance of 0.0:
4: Tiramisu, with distance of 0.0:
5: Rakhine Style Cucumber Salad, with distance of 1.0:
